In [ ]:
# For tips on running notebooks in Google Colab, see
# https://docs.pytorch.org/tutorials/beginner/colab
%matplotlib inline

Introduction to `torch.compile`
===============================

**Author:** William Wen


`torch.compile` is the latest method to speed up your PyTorch code!
`torch.compile` makes PyTorch code run faster by JIT-compiling PyTorch
code into optimized kernels, all while requiring minimal code changes.

In this tutorial, we cover basic `torch.compile` usage, and demonstrate
the advantages of `torch.compile` over previous PyTorch compiler
solutions, such as
[TorchScript](https://pytorch.org/docs/stable/jit.html) and [FX
Tracing](https://pytorch.org/docs/stable/fx.html#torch.fx.symbolic_trace).

**Contents**

::: {.contents local=""}
:::

**Required pip Dependencies**

-   `torch >= 2.0`
-   `torchvision`
-   `numpy`
-   `scipy`
-   `tabulate`

**System Requirements** - A C++ compiler, such as `g++` - Python
development package (`python-devel`/`python-dev`)


NOTE: a modern NVIDIA GPU (H100, A100, or V100) is recommended for this
tutorial in order to reproduce the speedup numbers shown below and
documented elsewhere.


In [1]:
import torch
import warnings

gpu_ok = False
if torch.cuda.is_available():
    device_cap = torch.cuda.get_device_capability()
    if device_cap in ((7, 0), (8, 0), (9, 0)):
        gpu_ok = True

if not gpu_ok:
    warnings.warn(
        "GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower "
        "than expected."
    )

/tmp/ipython-input-231343347.py:11: UserWarning: GPU is not NVIDIA V100, A100, or H100. Speedup numbers may be lower than expected.
  warnings.warn(


Basic Usage
===========

`torch.compile` is included in the latest PyTorch. Running TorchInductor
on GPU requires Triton, which is included with the PyTorch 2.0 nightly
binary. If Triton is still missing, try installing `torchtriton` via pip
(`pip install torchtriton --extra-index-url "https://download.pytorch.org/whl/nightly/cu117"`
for CUDA 11.7).

Arbitrary Python functions can be optimized by passing the callable to
`torch.compile`. We can then call the returned optimized function in
place of the original function.


In [2]:
def foo(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
opt_foo1 = torch.compile(foo)
print(opt_foo1(torch.randn(10, 10), torch.randn(10, 10)))

tensor([[ 1.5450e+00,  1.6511e+00,  1.3025e-01,  1.9594e+00,  1.4767e+00,
          1.4911e+00, -5.2126e-01, -8.0121e-01, -6.3500e-01,  1.7804e+00],
        [ 9.2160e-01,  1.3365e+00, -1.1250e-02,  1.7128e+00,  8.6725e-05,
          1.2389e+00,  4.1748e-01,  9.2930e-01,  1.4453e+00, -4.0526e-01],
        [ 1.5274e+00,  1.3973e+00,  2.2867e-01, -1.3317e+00,  1.6107e+00,
          8.7532e-01, -2.2581e-01,  1.8314e+00,  5.5317e-01,  1.3274e-01],
        [ 1.0966e-01,  1.8300e+00,  1.3707e+00,  5.7320e-02, -1.1489e-01,
          7.6217e-01,  9.0302e-01, -2.1708e-01, -3.2718e-01,  9.4489e-01],
        [ 1.3068e+00,  2.3594e-01,  1.5283e+00, -4.3143e-02,  9.7910e-01,
          1.4888e+00, -6.3476e-02,  3.1818e-01,  8.6624e-01,  1.3813e+00],
        [ 6.5304e-01,  9.6215e-01,  1.1121e+00,  1.5797e+00, -3.6471e-02,
          5.9483e-01,  2.6825e-01, -3.8146e-01,  2.8290e-01, -1.4828e-01],
        [ 1.3710e+00,  1.4516e-01, -5.3104e-02,  1.9894e+00,  1.5735e+00,
         -2.4207e-02, -2.7486e-0

Alternatively, we can decorate the function.


In [3]:
t1 = torch.randn(10, 10)
t2 = torch.randn(10, 10)

@torch.compile
def opt_foo2(x, y):
    a = torch.sin(x)
    b = torch.cos(y)
    return a + b
print(opt_foo2(t1, t2))

tensor([[ 1.7732,  0.3691,  0.2552, -0.3640,  1.0590,  1.7539,  0.3877,  0.9421,
          1.5616,  1.0058],
        [ 0.7166,  0.5926, -0.0889, -0.2178, -1.3075,  0.8030, -1.0373,  0.5751,
          1.4538,  1.0188],
        [ 1.4013,  1.1549, -0.5091,  0.8962,  1.0791,  0.4993,  1.0461,  0.9562,
          0.9175, -1.3197],
        [-0.1725,  0.9005,  0.5977,  0.5927,  1.5549,  0.0110,  0.1890,  1.8528,
         -1.2107, -0.4988],
        [-0.9087,  0.4290,  1.2854,  1.2222, -0.4186, -0.0373,  0.1208,  0.5796,
         -0.5790, -0.1019],
        [ 1.1605,  0.9795,  1.7620,  0.9011,  1.7396,  1.3510,  0.2098,  0.6857,
         -0.5174,  1.3387],
        [ 1.2042,  0.9970, -0.4190, -0.2182, -0.2630, -0.9949,  1.0849, -0.6055,
          1.5821,  1.8286],
        [-0.2155, -0.4102,  1.9005, -1.5212,  1.1366, -0.0046, -0.4107, -0.2908,
         -1.3449,  0.9613],
        [-0.6087, -0.3046,  0.1043, -0.2110,  0.8491,  1.3488,  0.2362, -0.4808,
          0.7507, -0.7170],
        [ 0.3986,  

We can also optimize `torch.nn.Module` instances.


In [4]:
t = torch.randn(10, 100)

class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lin = torch.nn.Linear(100, 10)

    def forward(self, x):
        return torch.nn.functional.relu(self.lin(x))

mod = MyModule()
mod.compile()
print(mod(t))
## or:
# opt_mod = torch.compile(mod)
# print(opt_mod(t))

tensor([[0.2307, 0.0061, 0.1172, 0.2177, 0.0000, 0.0000, 0.0000, 1.3746, 0.0000,
         0.0000],
        [0.0320, 0.0000, 0.0868, 0.0000, 0.0000, 0.0000, 1.2249, 0.3855, 0.0000,
         0.0000],
        [0.6496, 0.0000, 0.8511, 1.0356, 0.1513, 0.0000, 0.0000, 0.5793, 0.0000,
         0.0000],
        [0.0000, 0.3003, 0.0000, 0.2690, 0.3524, 0.3895, 0.0319, 0.2812, 0.0000,
         0.6048],
        [0.0000, 0.0949, 0.6786, 0.0000, 0.6974, 0.1081, 0.0000, 0.6810, 0.3419,
         0.0000],
        [0.3015, 0.0000, 0.0000, 0.0000, 0.7453, 0.1472, 0.2950, 0.5141, 0.1711,
         0.6255],
        [0.4007, 0.0000, 0.2488, 0.9309, 0.0000, 0.3030, 0.4395, 0.0000, 0.3464,
         0.1027],
        [0.5147, 0.0000, 0.0000, 0.0000, 0.0000, 0.1440, 0.6766, 0.3511, 0.0000,
         0.4167],
        [1.2033, 0.8359, 0.1258, 0.2013, 0.0000, 0.1264, 0.7995, 0.0000, 1.4419,
         0.0000],
        [0.0000, 0.1926, 0.0836, 0.4105, 0.0386, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000]], grad_fn=<

torch.compile and Nested Calls
==============================

Nested function calls within the decorated function will also be
compiled.


In [5]:
def nested_function(x):
    return torch.sin(x)

@torch.compile
def outer_function(x, y):
    a = nested_function(x)
    b = torch.cos(y)
    return a + b

print(outer_function(t1, t2))

tensor([[ 1.7732,  0.3691,  0.2552, -0.3640,  1.0590,  1.7539,  0.3877,  0.9421,
          1.5616,  1.0058],
        [ 0.7166,  0.5926, -0.0889, -0.2178, -1.3075,  0.8030, -1.0373,  0.5751,
          1.4538,  1.0188],
        [ 1.4013,  1.1549, -0.5091,  0.8962,  1.0791,  0.4993,  1.0461,  0.9562,
          0.9175, -1.3197],
        [-0.1725,  0.9005,  0.5977,  0.5927,  1.5549,  0.0110,  0.1890,  1.8528,
         -1.2107, -0.4988],
        [-0.9087,  0.4290,  1.2854,  1.2222, -0.4186, -0.0373,  0.1208,  0.5796,
         -0.5790, -0.1019],
        [ 1.1605,  0.9795,  1.7620,  0.9011,  1.7396,  1.3510,  0.2098,  0.6857,
         -0.5174,  1.3387],
        [ 1.2042,  0.9970, -0.4190, -0.2182, -0.2630, -0.9949,  1.0849, -0.6055,
          1.5821,  1.8286],
        [-0.2155, -0.4102,  1.9005, -1.5212,  1.1366, -0.0046, -0.4107, -0.2908,
         -1.3449,  0.9613],
        [-0.6087, -0.3046,  0.1043, -0.2110,  0.8491,  1.3488,  0.2362, -0.4808,
          0.7507, -0.7170],
        [ 0.3986,  

In the same fashion, when compiling a module all sub-modules and methods
within it, that are not in a skip list, are also compiled.


In [6]:
class OuterModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.inner_module = MyModule()
        self.outer_lin = torch.nn.Linear(10, 2)

    def forward(self, x):
        x = self.inner_module(x)
        return torch.nn.functional.relu(self.outer_lin(x))

outer_mod = OuterModule()
outer_mod.compile()
print(outer_mod(t))

tensor([[0.2420, 0.0904],
        [0.5552, 0.8287],
        [0.4470, 0.3854],
        [0.4659, 0.4388],
        [0.8060, 0.7300],
        [0.4278, 0.6427],
        [0.3320, 0.2014],
        [0.3366, 0.0000],
        [0.5014, 0.2860],
        [0.4942, 0.0000]], grad_fn=<CompiledFunctionBackward>)


We can also disable some functions from being compiled by using
`torch.compiler.disable`. Suppose you want to disable the tracing on
just the `complex_function` function, but want to continue the tracing
back in `complex_conjugate`. In this case, you can use
`torch.compiler.disable(recursive=False)` option. Otherwise, the default
is `recursive=True`.


In [7]:
def complex_conjugate(z):
    return torch.conj(z)

@torch.compiler.disable(recursive=False)
def complex_function(real, imag):
    # Assuming this function cause problems in the compilation
    z = torch.complex(real, imag)
    return complex_conjugate(z)

def outer_function():
    real = torch.tensor([2, 3], dtype=torch.float32)
    imag = torch.tensor([4, 5], dtype=torch.float32)
    z = complex_function(real, imag)
    return torch.abs(z)

# Try to compile the outer_function
try:
    opt_outer_function = torch.compile(outer_function)
    print(opt_outer_function())
except Exception as e:
    print("Compilation of outer_function failed:", e)

tensor([4.4721, 5.8310])


/usr/local/lib/python3.12/dist-packages/torch/_inductor/lowering.py:1890: UserWarning: Torchinductor does not support code generation for complex operators. Performance may be worse than eager.
  warnings.warn(


Best Practices and Recommendations
==================================

Behavior of `torch.compile` with Nested Modules and Function Calls

When you use `torch.compile`, the compiler will try to recursively
compile every function call inside the target function or module inside
the target function or module that is not in a skip list (such as
built-ins, some functions in the torch.\* namespace).

**Best Practices:**

1\. **Top-Level Compilation:** One approach is to compile at the highest
level possible (i.e., when the top-level module is initialized/called)
and selectively disable compilation when encountering excessive graph
breaks or errors. If there are still many compile issues, compile
individual subcomponents instead.

2\. **Modular Testing:** Test individual functions and modules with
`torch.compile` before integrating them into larger models to isolate
potential issues.

3\. **Disable Compilation Selectively:** If certain functions or
sub-modules cannot be handled by [torch.compile]{.title-ref}, use the
[torch.compiler.disable]{.title-ref} context managers to recursively
exclude them from compilation.

4\. **Compile Leaf Functions First:** In complex models with multiple
nested functions and modules, start by compiling the leaf functions or
modules first. For more information see [TorchDynamo APIs for
fine-grained
tracing](https://pytorch.org/docs/stable/torch.compiler_fine_grain_apis.html).

5.  **Prefer \`\`mod.compile()\`\` over \`\`torch.compile(mod)\`\`:**
    Avoids `_orig_` prefix issues in `state_dict`.

6\. **Use \`\`fullgraph=True\`\` to catch graph breaks:** Helps ensure
end-to-end compilation, maximizing speedup and compatibility with
`torch.export`.


Demonstrating Speedups
======================

Let\'s now demonstrate that using `torch.compile` can speed up real
models. We will compare standard eager mode and `torch.compile` by
evaluating and training a `torchvision` model on random data.

Before we start, we need to define some utility functions.


In [8]:
# Returns the result of running `fn()` and the time it took for `fn()` to run,
# in seconds. We use CUDA events and synchronization for the most accurate
# measurements.
def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000

# Generates random input and targets data for the model, where `b` is
# batch size.
def generate_data(b):
    return (
        torch.randn(b, 3, 128, 128).to(torch.float32).cuda(),
        torch.randint(1000, (b,)).cuda(),
    )

N_ITERS = 10

from torchvision.models import densenet121
def init_model():
    return densenet121().to(torch.float32).cuda()

First, let\'s compare inference.

Note that in the call to `torch.compile`, we have the additional `mode`
argument, which we will discuss below.


In [9]:
model = init_model()

# Reset since we are using a different mode.
import torch._dynamo
torch._dynamo.reset()

model_opt = torch.compile(model, mode="reduce-overhead")

inp = generate_data(16)[0]
with torch.no_grad():
    print("eager:", timed(lambda: model(inp))[1])
    print("compile:", timed(lambda: model_opt(inp))[1])

eager: 0.9170977172851562


W1006 02:16:45.297000 528 torch/_inductor/utils.py:1436] [0/0] Not enough SMs to use max_autotune_gemm mode


compile: 146.777046875


Notice that `torch.compile` takes a lot longer to complete compared to
eager. This is because `torch.compile` compiles the model into optimized
kernels as it executes. In our example, the structure of the model
doesn\'t change, and so recompilation is not needed. So if we run our
optimized model several more times, we should see a significant
improvement compared to eager.


In [10]:
eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, eager_time = timed(lambda: model(inp))
    eager_times.append(eager_time)
    print(f"eager eval time {i}: {eager_time}")

print("~" * 10)

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)[0]
    with torch.no_grad():
        _, compile_time = timed(lambda: model_opt(inp))
    compile_times.append(compile_time)
    print(f"compile eval time {i}: {compile_time}")
print("~" * 10)

import numpy as np
eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(eval) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager eval time 0: 0.035243999481201174
eager eval time 1: 0.034777118682861326
eager eval time 2: 0.03477471923828125
eager eval time 3: 0.03479142379760742
eager eval time 4: 0.024233983993530273
eager eval time 5: 0.024199167251586915
eager eval time 6: 0.02423561668395996
eager eval time 7: 0.023952863693237306
eager eval time 8: 0.023754911422729494
eager eval time 9: 0.02371251106262207
~~~~~~~~~~
compile eval time 0: 0.7716841430664062
compile eval time 1: 0.021178016662597655
compile eval time 2: 0.022683935165405275
compile eval time 3: 0.02191084861755371
compile eval time 4: 0.021217599868774414
compile eval time 5: 0.02134841537475586
compile eval time 6: 0.021748992919921876
compile eval time 7: 0.021311487197875977
compile eval time 8: 0.02087731170654297
compile eval time 9: 0.021416288375854492
~~~~~~~~~~
(eval) eager median: 0.024234800338745116, compile median: 0.021382351875305176, speedup: 1.1334019980624432x
~~~~~~~~~~


And indeed, we can see that running our model with `torch.compile`
results in a significant speedup. Speedup mainly comes from reducing
Python overhead and GPU read/writes, and so the observed speedup may
vary on factors such as model architecture and batch size. For example,
if a model\'s architecture is simple and the amount of data is large,
then the bottleneck would be GPU compute and the observed speedup may be
less significant.

You may also see different speedup results depending on the chosen
`mode` argument. The `"reduce-overhead"` mode uses CUDA graphs to
further reduce the overhead of Python. For your own models, you may need
to experiment with different modes to maximize speedup. You can read
more about modes
[here](https://pytorch.org/get-started/pytorch-2.0/#user-experience).

You may might also notice that the second time we run our model with
`torch.compile` is significantly slower than the other runs, although it
is much faster than the first run. This is because the
`"reduce-overhead"` mode runs a few warm-up iterations for CUDA graphs.

For general PyTorch benchmarking, you can try using
`torch.utils.benchmark` instead of the `timed` function we defined
above. We wrote our own timing function in this tutorial to show
`torch.compile`\'s compilation latency.

Now, let\'s consider comparing training.


In [11]:
model = init_model()
opt = torch.optim.Adam(model.parameters())

def train(mod, data):
    opt.zero_grad(True)
    pred = mod(data[0])
    loss = torch.nn.CrossEntropyLoss()(pred, data[1])
    loss.backward()
    opt.step()

eager_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, eager_time = timed(lambda: train(model, inp))
    eager_times.append(eager_time)
    print(f"eager train time {i}: {eager_time}")
print("~" * 10)

model = init_model()
opt = torch.optim.Adam(model.parameters())
train_opt = torch.compile(train, mode="reduce-overhead")

compile_times = []
for i in range(N_ITERS):
    inp = generate_data(16)
    _, compile_time = timed(lambda: train_opt(model, inp))
    compile_times.append(compile_time)
    print(f"compile train time {i}: {compile_time}")
print("~" * 10)

eager_med = np.median(eager_times)
compile_med = np.median(compile_times)
speedup = eager_med / compile_med
assert(speedup > 1)
print(f"(train) eager median: {eager_med}, compile median: {compile_med}, speedup: {speedup}x")
print("~" * 10)

eager train time 0: 0.481420166015625
eager train time 1: 0.09503395080566407
eager train time 2: 0.0805849609375
eager train time 3: 0.07024998474121094
eager train time 4: 0.06870604705810547
eager train time 5: 0.06474038696289063
eager train time 6: 0.06252006530761718
eager train time 7: 0.06675011444091797
eager train time 8: 0.06283967971801757
eager train time 9: 0.0663960952758789
~~~~~~~~~~


W1006 02:31:34.478000 528 torch/_logging/_internal.py:1154] [4/0] Profiler function <class 'torch.autograd.profiler.record_function'> will be ignored


compile train time 0: 275.41421875
compile train time 1: 7.0543662109375
compile train time 2: 0.08736771392822265
compile train time 3: 0.08148204803466796
compile train time 4: 0.07999324798583984
compile train time 5: 0.06726656341552735
compile train time 6: 0.052973247528076174
compile train time 7: 0.05121843338012695
compile train time 8: 0.04980358505249023
compile train time 9: 0.05253228759765625
~~~~~~~~~~


AssertionError: 

Again, we can see that `torch.compile` takes longer in the first
iteration, as it must compile the model, but in subsequent iterations,
we see significant speedups compared to eager.

We remark that the speedup numbers presented in this tutorial are for
demonstration purposes only. Official speedup values can be seen at the
[TorchInductor performance
dashboard](https://hud.pytorch.org/benchmark/compilers).


Comparison to TorchScript and FX Tracing
========================================

We have seen that `torch.compile` can speed up PyTorch code. Why else
should we use `torch.compile` over existing PyTorch compiler solutions,
such as TorchScript or FX Tracing? Primarily, the advantage of
`torch.compile` lies in its ability to handle arbitrary Python code with
minimal changes to existing code.

One case that `torch.compile` can handle that other compiler solutions
struggle with is data-dependent control flow (the `if x.sum() < 0:` line
below).


In [12]:
def f1(x, y):
    if x.sum() < 0:
        return -y
    return y

# Test that `fn1` and `fn2` return the same result, given
# the same arguments `args`. Typically, `fn1` will be an eager function
# while `fn2` will be a compiled function (torch.compile, TorchScript, or FX graph).
def test_fns(fn1, fn2, args):
    out1 = fn1(*args)
    out2 = fn2(*args)
    return torch.allclose(out1, out2)

inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)

TorchScript tracing `f1` results in silently incorrect results, since
only the actual control flow path is traced.


In [13]:
traced_f1 = torch.jit.trace(f1, (inp1, inp2))
print("traced 1, 1:", test_fns(f1, traced_f1, (inp1, inp2)))
print("traced 1, 2:", test_fns(f1, traced_f1, (-inp1, inp2)))

traced 1, 1: True
traced 1, 2: False


/tmp/ipython-input-4005623017.py:2: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() < 0:


FX tracing `f1` results in an error due to the presence of
data-dependent control flow.


In [14]:
import traceback as tb
try:
    torch.fx.symbolic_trace(f1)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipython-input-1451191637.py", line 3, in <cell line: 0>
    torch.fx.symbolic_trace(f1)
  File "/usr/local/lib/python3.12/dist-packages/torch/fx/_symbolic_trace.py", line 1326, in symbolic_trace
    graph = tracer.trace(root, concrete_args)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py", line 929, in _fn
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/fx/_symbolic_trace.py", line 850, in trace
    (self.create_arg(fn(*args)),),
                     ^^^^^^^^^
  File "/tmp/ipython-input-4005623017.py", line 2, in f1
    if x.sum() < 0:
       ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/fx/proxy.py", line 577, in __bool__
    return self.tracer.to_bool(self)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/fx/proxy.py", line 388, in to_b

If we provide a value for `x` as we try to FX trace `f1`, then we run
into the same problem as TorchScript tracing, as the data-dependent
control flow is removed in the traced function.


In [15]:
fx_f1 = torch.fx.symbolic_trace(f1, concrete_args={"x": inp1})
print("fx 1, 1:", test_fns(f1, fx_f1, (inp1, inp2)))
print("fx 1, 2:", test_fns(f1, fx_f1, (-inp1, inp2)))

fx 1, 1: True
fx 1, 2: False


/usr/local/lib/python3.12/dist-packages/torch/fx/_symbolic_trace.py:923: UserWarning: Was not able to add assertion to guarantee correct input x to specialized function. It is up to the user to make sure that your inputs match the inputs you specialized the function with.
  warnings.warn(


Now we can see that `torch.compile` correctly handles data-dependent
control flow.


In [16]:
# Reset since we are using a different mode.
torch._dynamo.reset()

compile_f1 = torch.compile(f1)
print("compile 1, 1:", test_fns(f1, compile_f1, (inp1, inp2)))
print("compile 1, 2:", test_fns(f1, compile_f1, (-inp1, inp2)))
print("~" * 10)

compile 1, 1: True
compile 1, 2: True
~~~~~~~~~~


TorchScript scripting can handle data-dependent control flow, but this
solution comes with its own set of problems. Namely, TorchScript
scripting can require major code changes and will raise errors when
unsupported Python is used.

In the example below, we forget TorchScript type annotations and we
receive a TorchScript error because the input type for argument `y`, an
`int`, does not match with the default argument type, `torch.Tensor`.


In [17]:
def f2(x, y):
    return x + y

inp1 = torch.randn(5, 5)
inp2 = 3

script_f2 = torch.jit.script(f2)
try:
    script_f2(inp1, inp2)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipython-input-895162831.py", line 9, in <cell line: 0>
    script_f2(inp1, inp2)
RuntimeError: f2() Expected a value of type 'Tensor (inferred)' for argument 'y' but instead found type 'int'.
Inferred 'y' to be of type 'Tensor' because it was not annotated with an explicit type.
Position: 1
Value: 3
Declaration: f2(Tensor x, Tensor y) -> Tensor
Cast error details: Unable to cast 3 to Tensor


However, `torch.compile` is easily able to handle `f2`.


In [18]:
compile_f2 = torch.compile(f2)
print("compile 2:", test_fns(f2, compile_f2, (inp1, inp2)))
print("~" * 10)

compile 2: True
~~~~~~~~~~


Another case that `torch.compile` handles well compared to previous
compilers solutions is the usage of non-PyTorch functions.


In [19]:
import scipy
def f3(x):
    x = x * 2
    x = scipy.fft.dct(x.numpy())
    x = torch.from_numpy(x)
    x = x * 2
    return x

TorchScript tracing treats results from non-PyTorch function calls as
constants, and so our results can be silently wrong.


In [20]:
inp1 = torch.randn(5, 5)
inp2 = torch.randn(5, 5)
traced_f3 = torch.jit.trace(f3, (inp1,))
print("traced 3:", test_fns(f3, traced_f3, (inp2,)))

traced 3: False


/tmp/ipython-input-123398134.py:4: TracerWarning: Converting a tensor to a NumPy array might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  x = scipy.fft.dct(x.numpy())
/tmp/ipython-input-123398134.py:5: TracerWarning: torch.from_numpy results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  x = torch.from_numpy(x)


TorchScript scripting and FX tracing disallow non-PyTorch function
calls.


In [21]:
try:
    torch.jit.script(f3)
except:
    tb.print_exc()

try:
    torch.fx.symbolic_trace(f3)
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipython-input-1991772511.py", line 2, in <cell line: 0>
    torch.jit.script(f3)
  File "/usr/local/lib/python3.12/dist-packages/torch/jit/_script.py", line 1443, in script
    ret = _script_impl(
          ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/jit/_script.py", line 1214, in _script_impl
    fn = torch._C._jit_script_compile(
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/_jit_internal.py", line 1238, in _try_get_dispatched_fn
    return boolean_dispatched.get(fn)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/weakref.py", line 452, in get
    return self.data.get(ref(key),default)
                         ^^^^^^^^
TypeError: cannot create weak reference to 'uarray._Function' object
Traceback (most recent call last):
  File "/tmp/ipython-input-1991772511.py", line 7, in <cell line: 0>
    torch.fx.symbolic_trace(f3)
  File "/usr/local/lib/python

In comparison, `torch.compile` is easily able to handle the non-PyTorch
function call.


In [22]:
compile_f3 = torch.compile(f3)
print("compile 3:", test_fns(f3, compile_f3, (inp2,)))

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/variables/functions.py:1481: UserWarning: Dynamo does not know how to trace the builtin `scipy.fft._pocketfft.pypocketfft.pybind11_detail_function_record_v1_system_libstdcpp_gxx_abi_1xxx_use_cxx11_abi_0.dct.` This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind).
If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround.
If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use `torch.compiler.allow_in_graph`.
  torch._dynamo.utils.warn_once(explanation + "\n" + "\n".join(hints))


InternalTorchDynamoError: AttributeError: 'str' object has no attribute 'IF_NEEDED'

from user code:
   File "/usr/local/lib/python3.12/dist-packages/scipy/fft/_realtransforms_backend.py", line 15, in torch_dynamo_resume_in__execute_at_12
    return xp.asarray(y)
  File "/usr/local/lib/python3.12/dist-packages/scipy/_lib/array_api_compat/numpy/_aliases.py", line 113, in asarray
    return np.array(obj, copy=copy, dtype=dtype, **kwargs)  # pyright: ignore

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


TorchDynamo and FX Graphs
=========================

One important component of `torch.compile` is TorchDynamo. TorchDynamo
is responsible for JIT compiling arbitrary Python code into [FX
graphs](https://pytorch.org/docs/stable/fx.html#torch.fx.Graph), which
can then be further optimized. TorchDynamo extracts FX graphs by
analyzing Python bytecode during runtime and detecting calls to PyTorch
operations.

Normally, TorchInductor, another component of `torch.compile`, further
compiles the FX graphs into optimized kernels, but TorchDynamo allows
for different backends to be used. In order to inspect the FX graphs
that TorchDynamo outputs, let us create a custom backend that outputs
the FX graph and simply returns the graph\'s unoptimized forward method.


In [23]:
from typing import List
def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("custom backend called with FX graph:")
    gm.graph.print_tabular()
    return gm.forward

# Reset since we are using a different backend.
torch._dynamo.reset()

opt_model = torch.compile(init_model(), backend=custom_backend)
opt_model(generate_data(16)[0])

custom backend called with FX graph:
opcode         name                                                                                                         target                                                                                                       args                                                                                                                                                                                                                                                                                                                                                                                                                                                     kwargs
-------------  -----------------------------------------------------------------------------------------------------------  -----------------------------------------------------------------------------------------------------------  -------------------------------------------------

tensor([[-0.1286, -0.7948,  0.1423,  ..., -0.2936,  0.0376,  0.0132],
        [-0.0925, -0.5208, -0.0995,  ..., -0.2743, -0.0212, -0.2851],
        [-0.1874, -0.3669, -0.0740,  ..., -0.1916, -0.1875, -0.1772],
        ...,
        [-0.1857, -0.6005, -0.0435,  ..., -0.1249, -0.0358, -0.0116],
        [-0.1472, -0.6281,  0.0725,  ..., -0.3083,  0.1127, -0.1344],
        [-0.0768, -0.3701, -0.0551,  ..., -0.1780, -0.0635, -0.1826]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

Using our custom backend, we can now see how TorchDynamo is able to
handle data-dependent control flow. Consider the function below, where
the line `if b.sum() < 0` is the source of data-dependent control flow.


In [24]:
def bar(a, b):
    x = a / (torch.abs(a) + 1)
    if b.sum() < 0:
        b = b * -1
    return x * b

opt_bar = torch.compile(bar, backend=custom_backend)
inp1 = torch.randn(10)
inp2 = torch.randn(10)
opt_bar(inp1, inp2)
opt_bar(inp1, -inp2)

custom backend called with FX graph:
opcode         name    target                                                  args         kwargs
-------------  ------  ------------------------------------------------------  -----------  --------
placeholder    l_a_    L_a_                                                    ()           {}
placeholder    l_b_    L_b_                                                    ()           {}
call_function  abs_1   <built-in method abs of type object at 0x7f4506ef74a0>  (l_a_,)      {}
call_function  add     <built-in function add>                                 (abs_1, 1)   {}
call_function  x       <built-in function truediv>                             (l_a_, add)  {}
call_method    sum_1   sum                                                     (l_b_,)      {}
call_function  lt      <built-in function lt>                                  (sum_1, 0)   {}
output         output  output                                                  ((lt, x),)   {}
cus

tensor([ 0.2063,  0.1797,  0.7211, -0.1127,  0.0848, -0.3400, -0.0989,  0.0971,
        -0.3263, -0.2934])

The output reveals that TorchDynamo extracted 3 different FX graphs
corresponding the following code (order may differ from the output
above):

1.  `x = a / (torch.abs(a) + 1)`
2.  `b = b * -1; return x * b`
3.  `return x * b`

When TorchDynamo encounters unsupported Python features, such as
data-dependent control flow, it breaks the computation graph, lets the
default Python interpreter handle the unsupported code, then resumes
capturing the graph.

Let\'s investigate by example how TorchDynamo would step through `bar`.
If `b.sum() < 0`, then TorchDynamo would run graph 1, let Python
determine the result of the conditional, then run graph 2. On the other
hand, if `not b.sum() < 0`, then TorchDynamo would run graph 1, let
Python determine the result of the conditional, then run graph 3.

This highlights a major difference between TorchDynamo and previous
PyTorch compiler solutions. When encountering unsupported Python
features, previous solutions either raise an error or silently fail.
TorchDynamo, on the other hand, will break the computation graph.

We can see where TorchDynamo breaks the graph by using
`torch._dynamo.explain`:


In [25]:
# Reset since we are using a different backend.
torch._dynamo.reset()
explain_output = torch._dynamo.explain(bar)(torch.randn(10), torch.randn(10))
print(explain_output)

Graph Count: 2
Graph Break Count: 1
Op Count: 5
Break Reasons:
  Break Reason 1:
    Reason: generic_jump TensorVariable()
    User Stack:
      <FrameSummary file /tmp/ipython-input-3263660924.py, line 3 in bar>
Ops per Graph:
  Ops 1:
    <built-in method abs of type object at 0x7f4506ef74a0>
    <built-in function add>
    <built-in function truediv>
    <built-in function lt>
  Ops 2:
    <built-in function mul>
Out Guards:
  Guard 1:
    Name: "L['a']"
    Source: local
    Create Function: TENSOR_MATCH
    Guard Types: ['TENSOR_MATCH']
    Code List: ["hasattr(L['a'], '_dynamo_dynamic_indices') == False"]
    Object Weakref: <weakref at 0x7f433e711a30; dead>
    Guarded Class Weakref: <weakref at 0x7f451c1050d0; to 'torch._C._TensorMeta' at 0x384369b0 (Tensor)>
  Guard 2:
    Name: ''
    Source: global
    Create Function: DEFAULT_DEVICE
    Guard Types: ['DEFAULT_DEVICE']
    Code List: ['utils_device.CURRENT_DEVICE == None']
    Object Weakref: None
    Guarded Class Weakref: 

In order to maximize speedup, graph breaks should be limited. We can
force TorchDynamo to raise an error upon the first graph break
encountered by using `fullgraph=True`:


In [26]:
opt_bar = torch.compile(bar, fullgraph=True)
try:
    opt_bar(torch.randn(10), torch.randn(10))
except:
    tb.print_exc()

Traceback (most recent call last):
  File "/tmp/ipython-input-3610564610.py", line 3, in <cell line: 0>
    opt_bar(torch.randn(10), torch.randn(10))
  File "/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py", line 745, in compile_wrapper
    raise e.with_traceback(None) from e.__cause__  # User compiler error
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
torch._dynamo.exc.Unsupported: Data-dependent branching
  Explanation: Detected data-dependent branching (e.g. `if my_tensor.sum() > 0:`). Dynamo does not support tracing dynamic control flow.
  Hint: This graph break is fundamental - it is unlikely that Dynamo will ever be able to trace through your code. Consider finding a workaround.
  Hint: Use `torch.cond` to express dynamic control flow.

  Developer debug context: attempted to jump with TensorVariable()


from user code:
   File "/tmp/ipython-input-3263660924.py", line 3, in bar
    if b.sum() < 0:

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (p

And below, we demonstrate that TorchDynamo does not break the graph on
the model we used above for demonstrating speedups.


In [27]:
opt_model = torch.compile(init_model(), fullgraph=True)
print(opt_model(generate_data(16)[0]))

tensor([[ 0.1748,  0.3107,  0.2403,  ..., -0.0005,  0.3335,  0.1878],
        [-0.0054,  0.1583,  0.0873,  ...,  0.0098,  0.1242, -0.0832],
        [ 0.0666,  0.2394,  0.2251,  ...,  0.0367,  0.1577,  0.0058],
        ...,
        [ 0.1483,  0.3717,  0.1451,  ..., -0.1594,  0.1310, -0.0144],
        [ 0.0490,  0.3453,  0.0891,  ..., -0.0628,  0.0832,  0.1443],
        [ 0.0398,  0.1468,  0.2055,  ..., -0.0505,  0.0752,  0.0751]],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>)


We can use `torch.export` (from PyTorch 2.1+) to extract a single,
exportable FX graph from the input PyTorch program. The exported graph
is intended to be run on different (i.e. Python-less) environments. One
important restriction is that the `torch.export` does not support graph
breaks. Please check [this
tutorial](https://pytorch.org/tutorials/intermediate/torch_export_tutorial.html)
for more details on `torch.export`.


Conclusion
==========

In this tutorial, we introduced `torch.compile` by covering basic usage,
demonstrating speedups over eager mode, comparing to previous PyTorch
compiler solutions, and briefly investigating TorchDynamo and its
interactions with FX graphs. We hope that you will give `torch.compile`
a try!
